# Open Alex Extraction and Matching with .search()

The goal of this Notebook is look up the PhD students (Authors) contained in the [cleaned](clean_data.ipynb) NARCIS dataset, and
1. Confirm they can be found in OpenAlex
2. Confirm their affiliation in NARCIS matches the one in OpenAlex
2. Confirm they wrote the associated PhD Thesis
3. Per author, look up all the contributors (i.e. potential first supervisors) that are listen in the NARCIS dataset and
    a. Find all authors that have worked for the same organization at the time the PhD thesis was published (within a 1 year window)
    b. xxx


The previous version of this notebook written by a Bachelor student was using the `.search_filter()` method of `pyalex`, which does not search alternate spellings of the specified name. In this notebook we are using `search_filter()`, which does not have that problem. See the example code [here](search_parameter_vs_search_filter.ipynb).

## 1. Setup

In [ ]:
#from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders, Concepts
import pyalex # importing full package seems to be the only way to call `pyalex.config.email = email_address`
import pandas as pd
from os import path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from src.unabbreviate_institutions import unabbreviate_institutions
from src.open_alex_helpers import AuthorRelations, find_phd_and_supervisors_in_row, get_supervisors_openalex_ids
from src.io_helpers import remove_illegal_title_characters
from src.clean_names_helpers import format_name_to_lastname_firstname
from src.config_selector import get_dataset_selector

# Initialize tqdm for progress bars
tqdm.pandas()

In [ ]:
# Create and display the selector
selector = get_dataset_selector()
selector.display()

In [ ]:
NROWS = selector.NROWS
use_dataset = selector.use_dataset
output_filename = selector.output_filename

print(f"Now using dataset: {use_dataset} with NROWS={NROWS}")

In [ ]:
# Specify PhD matching criterion
criterion="either"

# the contributor matching criterion right now is hard-coded to same 'institution at graduation'

Notebook settings

In [ ]:
# Automatically reloads any modules that are imported, 
# so that any changes made to the module files are reflected # without needing to restart the Jupyter kernel.
# load autoreload module
%load_ext autoreload
# mode 1 reloads only when an import statement is called. For production
# mode 2 reloads before execution of every cell
%autoreload 2

# limit the number of rows that are shown with printing data frames
pd.set_option('display.max_rows', 5)

Set contact email address to get to use the [polite pool](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication#the-polite-pool). Also, if you are on a premium plan, you can access the higher usage limit by using the associated email address.

In [ ]:
# Get contact email address from file
email_file_path = 'contact_email.txt'

if path.isfile(email_file_path):
    with open(email_file_path, 'r') as file:
        email_address = file.read().strip()

    # Assign the email address to the pyalex configuration
    pyalex.config.email = email_address

pyalex.config.email

## 2. Load datasets

### 2.1 Cleaned processed NARCIS dataset

In [ ]:
if NROWS == None:
    output_filename = 'data/output/matched_pairs_full.csv'
else:
    output_filename = f'data/output/matched_pairs_{NROWS}_rows.csv'

if use_dataset == 'biomedical_5y':
    # biomedical subset
    pubs_df = pd.read_csv('data/cleaned/pubs_biomedical5y.csv')
else:
    # general dataset
    pubs_df = pd.read_csv('data/cleaned/pubs.csv')

# Take a sample

if NROWS == None:
    n_sample = len(pubs_df)
else:
    n_sample = NROWS

pubs_df = pubs_df.sample(n=n_sample, random_state=42).reset_index(drop=True)

pubs_df

We need some additional data preparation and cleaning to make sure the dataset interacts well with OpenAlex.

In [ ]:
# replace institution abbreviation with names that can be found in OpenAlex
pubs_unabbrev_df = unabbreviate_institutions(pubs_df, 'institution')
pubs_unabbrev_df

# remove illegal characters from title that could trip up the API
pubs_unabbrev_df["title"] = pubs_unabbrev_df["title"].apply(remove_illegal_title_characters)
pubs_unabbrev_df


### 2.2 Priority supervisor list from ResponsibleSupervision pilot

This dataset was created during the Responsible Supervision pilot project, see [here](https://github.com/tamarinde/ResponsibleSupervision/tree/main/Pilot-responsible-supervision).

In [ ]:
repo_url = "https://github.com/tamarinde/ResponsibleSupervision/tree/main/Pilot-responsible-supervision/data/spreadsheets"
csv_path = "data/output/sups_pilot.csv"

try:
    # Attempt to read the supervisors in the pilot dataset from csv_path
    # If it fails, we get them again from GitHub
    supervisors_in_pilot_dataset = get_supervisors_openalex_ids(repo_url, csv_path)
    print("Unique Supervisors with OpenAlex IDs:")
    print(supervisors_in_pilot_dataset)
except Exception as e:
    print(f"An error occurred: {e}")

## 3. Extraction

In [ ]:
# set the dict to overwrite the default class attribute specified in src/open_alex_helpers.py
AuthorRelations.supervisors_in_pilot_dataset = supervisors_in_pilot_dataset

# Apply the function to each row
extraction_series = pubs_unabbrev_df.progress_apply(
    find_phd_and_supervisors_in_row,
    axis=1,
    args=(criterion,) # PhD matching criterion
)

# Concatenate all DataFrames into one
extraction_df = pd.concat(list(extraction_series), ignore_index=True)

extraction_df.to_csv(output_filename, index=False)

extraction_df

## 4. Analysis and Visualization

Load the extraction dataset from file in case we didn't run the extraction

In [ ]:
if 'extraction_df' not in locals() and 'extraction_df' not in globals():
    file_path = output_filename
    
    # Check if the file exists
    if path.exists(file_path):
        extraction_df = pd.read_csv(file_path)
        print(f"Read `extraction_df` from {file_path}")
    else:
        raise FileNotFoundError(f"File not found: {file_path}")
    
extraction_df

Get PhDs that we could not find in OpenAlex.

In [ ]:
# Step 1: Filter extraction_df for rows with phd_id = NaN
extraction_none_df = extraction_df.query("phd_id != phd_id")

# Step 2: Filter pubs_unabbrev_df for matching phd_names; then sort and export
pubs_phd_not_confirmed_df = (
    pubs_unabbrev_df
    .query("phd_name in @extraction_none_df.phd_name")
    .sort_values(by=["year", "institution"])   # sort by multiple columns
)

# Export to CSV without the DataFrame index
pubs_phd_not_confirmed_df.to_csv("data/output/phds_not_confirmed.csv", index=False)

pubs_phd_not_confirmed_df


In [ ]:
# Ensure only one row per PhD for counting how they were confirmed
count_phds_df = extraction_df[['phd_name', 'phd_match_by']].drop_duplicates(subset='phd_name')

n_phds = len(count_phds_df)
n_confirmed_phds = len(count_phds_df.query('phd_match_by.notna()'))

# Replace NaN in 'phd_match_by' with 'Not confirmed'
count_phds_df['phd_match_by'] = count_phds_df['phd_match_by'].fillna('Not confirmed')

# Count how many PhDs per match type
match_phds_counts = count_phds_df['phd_match_by'].value_counts()

# Create a bar plot
ax = match_phds_counts.plot(kind='bar')

# Add count labels on top of each bar
ax.bar_label(ax.containers[0], label_type='edge')

# Add a legend specifying the matching criterion
ax.legend([f'PhD matching by "{criterion}"'], loc="upper right")

# Add labels and title for clarity
plt.xlabel("Match Type")
plt.ylabel("Number of PhDs")
plt.suptitle("PhD Matching Confirmation by Type",fontsize=12) # title
plt.title(f"Confirmed {n_confirmed_phds} out of {n_phds} PhDs. That is {round(n_confirmed_phds/n_phds*100, 2)}%", fontsize=10) # subtitle

# Display the plot
plt.show()

In [ ]:
# filter out rows where 'phd_id' is Na, so that we only look at PhDs we could confirm
count_contrib_df = extraction_df.query('phd_id.notna()')[['contributor_name', 'sup_match_by']]

# Replace NaN in 'sup_match_by' with 'Not confirmed'
count_contrib_df['sup_match_by'] = count_contrib_df['sup_match_by'].fillna('Not confirmed')

# Count how many contributors per match type
match_contrib_counts = count_contrib_df['sup_match_by'].value_counts()

# Create a bar plot
ax = match_contrib_counts.plot(kind='bar', color='#FFD54F')

# Add count labels on top of each bar
ax.bar_label(ax.containers[0], label_type='edge')

# Add labels and title for clarity
plt.xlabel("Match Type")
plt.ylabel("Number of contributors")
plt.suptitle("Contributor Matching Confirmation by Type", fontsize=12) # title
plt.title(f"Only considering the {n_confirmed_phds} PhDs we could confirm", fontsize=10) # subtile

# Display the plot
plt.show()